# MedQuAD Medical Chatbot with BioGPT Diagnosis

**Hybrid AI-powered medical diagnosis system** combining:

**Tier 1: Common Illness Detection** (10 most common conditions)
- Instant symptom-set matching
- Curated medical information
- Covers 80%+ of typical queries

**Tier 2: BioGPT + MedQuAD** (5000+ rare conditions)
- **BioGPT**: Accurate disease diagnosis (trained on 15M PubMed abstracts)
- **MedQuAD**: Patient-friendly descriptions (extracted from medical Q&A)

**Pipeline:**
1. Symptoms → BioGPT diagnosis → Identifies specific disease
2. Disease → MedQuAD extraction → Patient-friendly description
3. Best of both: Medical accuracy + Readability


In [1]:
# Install dependencies (run this if needed)
# !pip -q install sentence-transformers transformers --upgrade

## 1) Setup & Data Load

In [2]:
import os, re, math, json
import numpy as np
import pandas as pd
from collections import Counter, defaultdict

MEDQUAD_PATH = "medquad.csv"
COMMON_ILLNESSES_PATH = "common_illnesses.json"

df = pd.read_csv(MEDQUAD_PATH)
print(df.head(3))
print("\nColumns:", list(df.columns))
print("Rows:", len(df))

                              question  \
0             What is (are) Glaucoma ?   
1               What causes Glaucoma ?   
2  What are the symptoms of Glaucoma ?   

                                              answer           source  \
0  Glaucoma is a group of diseases that can damag...  NIHSeniorHealth   
1  Nearly 2.7 million people have glaucoma, a lea...  NIHSeniorHealth   
2  Symptoms of Glaucoma  Glaucoma can develop in ...  NIHSeniorHealth   

  focus_area  
0   Glaucoma  
1   Glaucoma  
2   Glaucoma  

Columns: ['question', 'answer', 'source', 'focus_area']
Rows: 16412


## 2) Load Common Illnesses Dataset

In [3]:
# Load common illnesses for fast matching
with open(COMMON_ILLNESSES_PATH, 'r') as f:
    common_conditions = json.load(f)

print("Common illnesses loaded:")
for key, value in common_conditions.items():
    print(f"  - {value['full_name']} ({len(value['symptoms']['primary'])} primary symptoms)")

Common illnesses loaded:
  - Hypertension (7 primary symptoms)
  - Upper Respiratory Tract Infection (6 primary symptoms)
  - Depression or Anxiety (7 primary symptoms)
  - Back Pain (6 primary symptoms)
  - Arthritis (5 primary symptoms)
  - Dermatitis (6 primary symptoms)
  - Acute Otitis Media (6 primary symptoms)
  - Diabetes (6 primary symptoms)
  - Urinary Tract Infection (6 primary symptoms)
  - Allergies (6 primary symptoms)


## 3) Text Preprocessing
We concatenate each entry's `question + answer` into a single searchable passage and keep the `focus_area` (disease/topic).

In [4]:
def clean_text(t: str) -> str:
    if not isinstance(t, str):
        return ""
    t = re.sub(r"\s+", " ", t)
    return t.strip()

df['question'] = df['question'].astype(str).apply(clean_text)
df['answer']   = df['answer'].astype(str).apply(clean_text)
df['focus_area'] = df['focus_area'].astype(str).apply(lambda x: x.strip())
df['text'] = (df['question'] + " \n " + df['answer']).str.strip()

unique_diseases = sorted(df['focus_area'].dropna().unique().tolist())
print(f"Total unique diseases in MedQuAD: {len(unique_diseases)}")
print(f"Sample diseases: {unique_diseases[:10]}")

Total unique diseases in MedQuAD: 5127
Sample diseases: ['11-beta-hydroxylase deficiency', '15q11.2 microdeletion', '15q13.3 microdeletion', '15q13.3 microdeletion syndrome', '15q13.3 microduplication syndrome', '15q24 microdeletion', '16p11.2 deletion syndrome', '16q24.3 microdeletion syndrome', '17 alpha-hydroxylase/17,20-lyase deficiency', '17-alpha-hydroxylase deficiency']


## 4) Common Illness Fast Checker
This function performs quick symptom-set matching for the 10 most common conditions.

In [5]:
def normalize_symptom(symptom):
    """Normalize symptom for better matching"""
    return symptom.lower().strip()

def check_common_conditions(patient_symptoms):
    """
    Fast check against common illnesses using symptom-set matching.
    
    Returns:
        tuple: (search_strategy, match_info)
        - search_strategy: 'common_illness', 'hybrid_search', or 'full_medquad'
        - match_info: dict with match details or None
    """
    # Normalize patient symptoms
    normalized_symptoms = [normalize_symptom(s) for s in patient_symptoms]
    normalized_set = set(normalized_symptoms)
    
    matches = {}
    
    for condition_key, data in common_conditions.items():
        # Normalize all condition symptoms
        primary_symptoms = [normalize_symptom(s) for s in data['symptoms']['primary']]
        secondary_symptoms = [normalize_symptom(s) for s in data['symptoms']['secondary']]
        
        # Calculate overlaps
        primary_matches = normalized_set & set(primary_symptoms)
        secondary_matches = normalized_set & set(secondary_symptoms)
        
        # Score: primary symptoms worth 2 points, secondary worth 1
        total_score = len(primary_matches) * 2 + len(secondary_matches)
        
        # Only consider if minimum match threshold is met
        if len(primary_matches) >= data['min_match'] or total_score >= data['min_match'] * 2:
            matches[condition_key] = {
                'full_name': data['full_name'],
                'score': total_score,
                'primary_matched': list(primary_matches),
                'secondary_matched': list(secondary_matches),
                'total_matched': len(primary_matches) + len(secondary_matches)
            }
    
    if not matches:
        return 'full_medquad', None
    
    # Get best match
    best_condition = max(matches.items(), key=lambda x: x[1]['score'])
    best_score = best_condition[1]['score']
    
    # Decision thresholds
    HIGH_CONFIDENCE = 4  # Strong match - use common illness result
    LOW_CONFIDENCE = 2   # Weak match - search both datasets
    
    if best_score >= HIGH_CONFIDENCE:
        return 'common_illness', best_condition
    elif best_score >= LOW_CONFIDENCE:
        return 'hybrid_search', best_condition
    else:
        return 'full_medquad', None

# Test the common illness checker
test_symptoms = ['sneezing', 'hoarse voice', 'nasal congestion']
strategy, match = check_common_conditions(test_symptoms)
print(f"\nTest symptoms: {test_symptoms}")
print(f"Strategy: {strategy}")
if match:
    print(f"Best match: {match[1]['full_name']} (score: {match[1]['score']})")
    print(f"Matched symptoms: {match[1]['primary_matched'] + match[1]['secondary_matched']}")


Test symptoms: ['sneezing', 'hoarse voice', 'nasal congestion']
Strategy: common_illness
Best match: Upper Respiratory Tract Infection (score: 5)
Matched symptoms: ['sneezing', 'nasal congestion', 'hoarse voice']


## 5) Embeddings & Retriever
Use a biomedical sentence‑transformer for better domain coverage. You can switch to a smaller general model if you prefer speed.

In [6]:
from sentence_transformers import SentenceTransformer, util
import torch

# You can swap in a general model like 'sentence-transformers/all-MiniLM-L6-v2' for speed
EMBED_MODEL = "pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb"
embedder = SentenceTransformer(EMBED_MODEL)

C:\Users\USER\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Encode corpus (run once, then load from cache)
corpus_texts = df['text'].tolist()
print("Encoding corpus...")
corpus_embs = embedder.encode(corpus_texts, batch_size=64, convert_to_tensor=True, show_progress_bar=True)

# Save both embeddings and texts
torch.save({
    'texts': corpus_texts,
    'embeddings': corpus_embs
}, 'corpus_embeddings.pt')

In [8]:
# Load pre-computed embeddings
data = torch.load('corpus_embeddings.pt')
corpus_texts = data['texts']
corpus_embs = data['embeddings']
print(f"Loaded {len(corpus_texts)} embeddings")

Loaded 16412 embeddings


## 6) Disease Prediction from Symptoms (Full MedQuAD Search)

## 6a) BioGPT-Based Disease Diagnosis

**NEW APPROACH**: Use BioGPT's medical knowledge to directly diagnose diseases from symptoms.

**How it works:**
1. BioGPT receives symptoms and suggests likely diagnoses
2. We match BioGPT's suggestion to the closest disease in MedQuAD database
3. Once matched, we use extractive approach to get descriptions from MedQuAD

**Benefits:**
- BioGPT's medical knowledge can identify specific conditions
- More accurate than pure semantic search for rare diseases
- Keeps patient-friendly descriptions from MedQuAD

In [9]:
from transformers import BioGptForCausalLM, BioGptTokenizer
import torch

# Load BioGPT for disease diagnosis
print("Loading BioGPT model for disease diagnosis...")
biogpt_model = BioGptForCausalLM.from_pretrained("microsoft/biogpt")
biogpt_tokenizer = BioGptTokenizer.from_pretrained("microsoft/biogpt")

device = "cuda" if torch.cuda.is_available() else "cpu"
biogpt_model = biogpt_model.to(device)
print(f"BioGPT loaded on {device}")

def diagnose_with_biogpt(symptoms, max_attempts=2):
    """
    Use BioGPT to identify disease from symptoms.
    Returns the disease name(s) suggested by BioGPT.
    """
    # Create clinical prompt
    symptom_text = ', '.join(symptoms)
    prompt = f"A patient presents with {symptom_text}. The most likely diagnosis is"
    
    try:
        inputs = biogpt_tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(device)
        
        with torch.no_grad():
            outputs = biogpt_model.generate(
                **inputs,
                max_length=inputs['input_ids'].shape[1] + 50,  # Short completion
                num_beams=5,
                early_stopping=True,
                no_repeat_ngram_size=2,
                temperature=0.7,
                do_sample=False,
                pad_token_id=biogpt_tokenizer.eos_token_id,
                num_return_sequences=3  # Get top 3 possibilities
            )
        
        # Decode all candidates
        candidates = []
        for output in outputs:
            generated_text = biogpt_tokenizer.decode(output, skip_special_tokens=True)
            
            # Extract the disease name after "diagnosis is"
            if "diagnosis is" in generated_text:
                disease_part = generated_text.split("diagnosis is", 1)[1].strip()
                # Get first sentence or phrase (up to period or comma)
                disease = disease_part.split('.')[0].split(',')[0].strip()
                if disease and len(disease) < 100:  # Reasonable length
                    candidates.append(disease)
        
        return candidates if candidates else None
    
    except Exception as e:
        print(f"BioGPT diagnosis error: {e}")
        return None

def find_closest_medquad_disease(biogpt_disease, top_n=5):
    """
    Find the closest matching disease in MedQuAD database.
    Uses fuzzy matching since BioGPT might use slightly different terminology.
    """
    if not biogpt_disease:
        return None
    
    biogpt_lower = biogpt_disease.lower()
    
    # Get all unique diseases from MedQuAD
    all_diseases = df['focus_area'].unique().tolist()
    
    # Try exact match first
    if biogpt_disease in all_diseases:
        return biogpt_disease
    
    # Try case-insensitive match
    for disease in all_diseases:
        if disease.lower() == biogpt_lower:
            return disease
    
    # Try partial match (BioGPT disease is substring of MedQuAD disease)
    partial_matches = []
    for disease in all_diseases:
        if biogpt_lower in disease.lower() or disease.lower() in biogpt_lower:
            partial_matches.append(disease)
    
    if partial_matches:
        return partial_matches[0]  # Return best partial match
    
    # Last resort: use semantic search with BioBERT embeddings
    query_emb = embedder.encode([biogpt_disease], convert_to_tensor=True)
    disease_texts = [f"{d}" for d in all_diseases]
    disease_embs = embedder.encode(disease_texts, convert_to_tensor=True)
    
    from sentence_transformers import util
    hits = util.semantic_search(query_emb, disease_embs, top_k=top_n)[0]
    
    if hits:
        best_match_idx = hits[0]['corpus_id']
        return all_diseases[best_match_idx]
    
    return None

# Test the BioGPT diagnosis
test_symptoms = ['muscle weakness', 'difficulty swallowing', 'double vision']
print(f"\nTesting BioGPT diagnosis with: {test_symptoms}")
biogpt_candidates = diagnose_with_biogpt(test_symptoms)
if biogpt_candidates:
    print(f"BioGPT suggested: {biogpt_candidates}")
    for candidate in biogpt_candidates[:2]:  # Try top 2
        matched = find_closest_medquad_disease(candidate)
        if matched:
            print(f"  '{candidate}' → Matched to MedQuAD disease: '{matched}'")
            break
else:
    print("BioGPT diagnosis failed")

Loading BioGPT model for disease diagnosis...


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


BioGPT loaded on cpu

Testing BioGPT diagnosis with: ['muscle weakness', 'difficulty swallowing', 'double vision']
BioGPT suggested: ['myasthenia gravis', 'Guillain-Barré syndrome', 'Guillain-Barré syndrome (GBS)']
  'myasthenia gravis' → Matched to MedQuAD disease: 'myasthenia gravis'


In [10]:
def predict_disease_from_symptoms(symptoms, top_k=50):
    """
    Full MedQuAD semantic search for disease prediction.
    Used when common illness check doesn't find a strong match.
    """
    query = "Patient presents with " + ", ".join(symptoms)
    q_emb = embedder.encode([query], convert_to_tensor=True)
    hits = util.semantic_search(q_emb, corpus_embs, top_k=top_k)[0]
    
    # Count diseases by focus_area
    disease_scores = Counter()
    for h in hits:
        idx = h['corpus_id']
        score = h['score']
        disease = df.iloc[idx]['focus_area']
        disease_scores[disease] += score
    
    ranked = disease_scores.most_common(10)
    return ranked, hits

# Test full search
test_symptoms = ['chest pain', 'shortness of breath', 'fatigue']
ranked_diseases, hits = predict_disease_from_symptoms(test_symptoms)
print(f"\nTest symptoms: {test_symptoms}")
print(f"Top 3 disease candidates from full search:")
for disease, score in ranked_diseases[:3]:
    print(f"  {disease}: {score:.3f}")


Test symptoms: ['chest pain', 'shortness of breath', 'fatigue']
Top 3 disease candidates from full search:
  Heart Failure: 3.675
  Respiratory Failure: 2.352
  Primary Myelofibrosis: 1.185


## 7) Retrieve Facts for Disease

In [11]:
def retrieve_facts_for_disease(disease, top_k=12):
    """
    Retrieve relevant Q&A facts for a specific disease from MedQuAD.
    """
    # Restrict to entries for this focus_area
    mask = (df['focus_area'] == disease)
    subset = df[mask]
    if subset.empty:
        return []
    
    texts = subset['text'].tolist()
    embs = embedder.encode(texts, convert_to_tensor=True)
    
    # Use disease name as query to pull the most central Q&A for this topic
    q_emb = embedder.encode([disease], convert_to_tensor=True)
    hits = util.semantic_search(q_emb, embs, top_k=min(top_k, len(texts)))[0]
    
    picked = []
    sub_q = subset['question'].tolist()
    sub_a = subset['answer'].tolist()
    for h in hits:
        i = h['corpus_id']
        picked.append({
            'question': sub_q[i],
            'answer': sub_a[i]
        })
    return picked

## 8) Information Extraction Functions
Extract relevant medical information directly from MedQuAD Q&A pairs. This approach ensures accuracy by using actual medical database content rather than generating new text.

In [12]:
def extract_description(disease, facts, max_sentences=5):
    """
    Extract description by finding answers to 'What is' questions.
    """
    if not facts:
        return f"No detailed information available for {disease} in the medical database."
    
    description_keywords = ['what is', 'what are', 'define', 'description', 'overview']
    relevant_answers = []
    
    # First pass: look for definition/description questions
    for fact in facts[:10]:  # Check first 10
        q_lower = fact['question'].lower()
        if any(keyword in q_lower for keyword in description_keywords):
            answer = fact['answer'].strip()
            if len(answer) > 50:  # Only substantial answers
                relevant_answers.append(answer)
                if len(relevant_answers) >= 2:  # Get top 2 description answers
                    break
    
    # If no specific description found, use first substantial answer
    if not relevant_answers:
        for fact in facts[:3]:
            answer = fact['answer'].strip()
            if len(answer) > 50:
                relevant_answers.append(answer)
                break
    
    if not relevant_answers:
        return f"Limited information available for {disease}."
    
    # Combine and clean up
    description = ' '.join(relevant_answers[:2])  # Max 2 answers
    
    # Limit to max_sentences
    sentences = [s.strip() + '.' for s in description.split('.') if s.strip()]
    if len(sentences) > max_sentences:
        sentences = sentences[:max_sentences]
    
    return ' '.join(sentences)

def extract_recommendations(disease, facts):
    """
    Extract recommendations by finding answers about prevention, management, or what to avoid.
    """
    if not facts:
        return "Consult with a healthcare provider for personalized recommendations."
    
    recommendation_keywords = [
        'prevent', 'avoid', 'should not', 'do not', 'management',
        'lifestyle', 'precaution', 'risk', 'complication', 'warning'
    ]
    
    relevant_answers = []
    
    # Look for recommendation-related questions
    for fact in facts:
        q_lower = fact['question'].lower()
        if any(keyword in q_lower for keyword in recommendation_keywords):
            answer = fact['answer'].strip()
            if len(answer) > 30:  # Only substantial answers
                relevant_answers.append(answer)
                if len(relevant_answers) >= 3:  # Get top 3
                    break
    
    if not relevant_answers:
        # Fallback: look for treatment or outlook questions
        fallback_keywords = ['treatment', 'outlook', 'prognosis', 'care']
        for fact in facts:
            q_lower = fact['question'].lower()
            if any(keyword in q_lower for keyword in fallback_keywords):
                answer = fact['answer'].strip()
                if len(answer) > 30:
                    relevant_answers.append(answer)
                    if len(relevant_answers) >= 2:
                        break
    
    if not relevant_answers:
        return "Consult with a healthcare provider for specific guidance on managing this condition."
    
    # Combine recommendations
    recommendations = ' '.join(relevant_answers[:3])  # Max 3 answers
    
    # Limit length
    sentences = [s.strip() + '.' for s in recommendations.split('.') if s.strip()]
    if len(sentences) > 6:
        sentences = sentences[:6]
    
    return ' '.join(sentences)


## 9) Integrated Two‑Tier Pipeline
This function combines common illness checking with full MedQuAD search.

In [13]:
def diagnose_and_answer(symptoms, use_biogpt=True, topk_candidates=50, verbose=True):
    """
    Two-tier diagnosis system with BioGPT option:
    1. Check common illnesses first (fast)
    2. Use BioGPT for disease diagnosis OR fall back to MedQuAD search
    3. Extract descriptions from MedQuAD (patient-friendly)
    
    Args:
        symptoms: list of symptom strings
        use_biogpt: if True, use BioGPT for diagnosis; if False, use MedQuAD semantic search
        topk_candidates: number of candidates for fallback MedQuAD search
        verbose: if True, print search strategy details
    """
    # Stage 0: Common illness fast check
    strategy, common_match = check_common_conditions(symptoms)
    
    if verbose:
        print(f"\n🔍 Search Strategy: {strategy.upper()}")
    
    disease = None
    confidence = None
    description = None
    recommendation = None
    
    if strategy == 'common_illness':
        # Strong match with common illness - use curated info
        condition_key = common_match[0]
        condition_data = common_conditions[condition_key]
        disease = condition_data['full_name']
        description = condition_data.get('description', '')
        recommendation = condition_data.get('recommendation', '')
        confidence = 'high'
        
        if verbose:
            print(f"✓ Common illness detected: {disease}")
            print(f"  Matched {common_match[1]['total_matched']} symptoms (score: {common_match[1]['score']})")
            print(f"  Using curated medical information")
    
    elif strategy == 'hybrid_search':
        # Partial match - compare common illness vs full search
        condition_key = common_match[0]
        condition_data = common_conditions[condition_key]
        common_disease = condition_data['full_name']
        
        if verbose:
            print(f"⚠ Partial match: {common_disease} (score: {common_match[1]['score']})")
            print(f"  Comparing with {'BioGPT' if use_biogpt else 'MedQuAD semantic'} search...")
        
        # Search using chosen method
        if use_biogpt:
            biogpt_candidates = diagnose_with_biogpt(symptoms)
            if biogpt_candidates:
                for candidate in biogpt_candidates[:2]:
                    matched_disease = find_closest_medquad_disease(candidate)
                    if matched_disease:
                        full_search_disease = matched_disease
                        if verbose:
                            print(f"  BioGPT suggested: '{candidate}' → '{matched_disease}'")
                        break
                else:
                    full_search_disease = None
            else:
                full_search_disease = None
        else:
            ranked, _ = predict_disease_from_symptoms(symptoms, top_k=topk_candidates)
            full_search_disease = ranked[0][0] if ranked else None
        
        # Decide which to use
        if full_search_disease:
            if verbose:
                print(f"  Full search result: {full_search_disease}")
            
            # Prefer common illness if it's the same or similar
            if common_disease.lower() in full_search_disease.lower() or full_search_disease.lower() in common_disease.lower():
                disease = common_disease
                description = condition_data.get('description', '')
                recommendation = condition_data.get('recommendation', '')
                if verbose:
                    print(f"  → Using common illness: {disease}")
            else:
                disease = full_search_disease
                if verbose:
                    print(f"  → Using full search result: {disease}")
                    print(f"  Extracting from MedQuAD...")
                facts = retrieve_facts_for_disease(disease, top_k=12)
                description = extract_description(disease, facts)
                recommendation = extract_recommendations(disease, facts)
            confidence = 'medium'
        else:
            disease = common_disease
            description = condition_data.get('description', '')
            recommendation = condition_data.get('recommendation', '')
            confidence = 'medium'
    
    else:  # full_medquad or full_biogpt
        # No common illness match
        if verbose:
            print(f"  No common illness match.")
        
        if use_biogpt:
            # Use BioGPT for diagnosis
            if verbose:
                print(f"  Using BioGPT for diagnosis...")
            
            biogpt_candidates = diagnose_with_biogpt(symptoms)
            if biogpt_candidates:
                # Try to match to MedQuAD
                disease = None
                for candidate in biogpt_candidates:
                    matched_disease = find_closest_medquad_disease(candidate)
                    if matched_disease:
                        disease = matched_disease
                        if verbose:
                            print(f"  BioGPT diagnosis: '{candidate}'")
                            print(f"  Matched to MedQuAD: '{disease}'")
                        break
                
                if not disease and biogpt_candidates:
                    # Use first BioGPT suggestion even if not perfect match
                    disease = biogpt_candidates[0]
                    if verbose:
                        print(f"  BioGPT diagnosis: '{disease}' (no exact MedQuAD match)")
            
            # Fallback to MedQuAD search if BioGPT fails
            if not disease:
                if verbose:
                    print(f"  BioGPT failed, falling back to MedQuAD semantic search...")
                ranked, _ = predict_disease_from_symptoms(symptoms, top_k=topk_candidates)
                if not ranked:
                    return {"error": "No disease candidates found."}
                disease = ranked[0][0]
                if verbose:
                    print(f"  MedQuAD top candidate: {disease}")
        else:
            # Use MedQuAD semantic search
            if verbose:
                print(f"  Using MedQuAD semantic search...")
            ranked, _ = predict_disease_from_symptoms(symptoms, top_k=topk_candidates)
            if not ranked:
                return {"error": "No disease candidates found."}
            disease = ranked[0][0]
            if verbose:
                print(f"  Top candidate: {disease}")
        
        # Extract from MedQuAD
        if verbose:
            print(f"  Extracting information from MedQuAD...")
        
        facts = retrieve_facts_for_disease(disease, top_k=12)
        if not facts:
            return {
                "disease": disease,
                "confidence": 'low',
                "strategy": 'biogpt' if use_biogpt else strategy,
                "warning": "No facts found in MedQuAD for this disease."
            }
        
        description = extract_description(disease, facts)
        recommendation = extract_recommendations(disease, facts)
        confidence = 'medium'
    
    # Final result
    if verbose:
        print(f"\n📋 Medical information compiled for {disease}")
    
    result = {
        "disease": disease,
        "confidence": confidence,
        "strategy": 'biogpt' if (use_biogpt and strategy == 'full_medquad') else strategy,
        "description": description,
        "recommendation": recommendation,
    }
    
    return result

## 10) Demo Examples

Test cases covering common to rare conditions:
1. Common illness (fast path)
2. Rare neuromuscular condition
3. Autoimmune disease
4. Endocrine disorder
5. Infectious disease

In [15]:
# Example 1: Common Upper Respiratory Infection
print("=" * 70)
print("Example 1: Common Cold (Should use Tier 1 - Fast Path)")
print("=" * 70)

symptoms_1 = ['itchy skin']
result_1 = diagnose_and_answer(symptoms_1, verbose=True)

print("\n📊 Results:")
print(f"Disease: {result_1['disease']}")
print(f"Confidence: {result_1['confidence']}")
print(f"Strategy: {result_1['strategy']}")
print(f"\nDescription: {result_1['description']}")
print(f"\nRecommendation: {result_1['recommendation']}")


Example 1: Common Cold (Should use Tier 1 - Fast Path)

🔍 Search Strategy: FULL_MEDQUAD
  No common illness match.
  Using BioGPT for diagnosis...


KeyboardInterrupt: 

In [ ]:
# Example 2: Rare Neuromuscular Disease
print("\n" + "=" * 70)
print("Example 2: Myasthenia Gravis (Rare - BioGPT Diagnosis)")
print("=" * 70)

symptoms_2 = ['muscle weakness', 'difficulty swallowing', 'double vision', 'drooping eyelids']
result_2 = diagnose_and_answer(symptoms_2, verbose=True)

print("\n📊 Results:")
print(f"Disease: {result_2['disease']}")
print(f"Confidence: {result_2['confidence']}")
print(f"\nDescription: {result_2['description']}")
print(f"\nRecommendation: {result_2['recommendation']}")



Example 2: Myasthenia Gravis (Rare - BioGPT Diagnosis)

🔍 Search Strategy: FULL_MEDQUAD
  No common illness match.
  Using BioGPT for diagnosis...
  BioGPT diagnosis: 'myasthenia gravis'
  Matched to MedQuAD: 'myasthenia gravis'
  Extracting information from MedQuAD...

📋 Medical information compiled for myasthenia gravis

📊 Results:
Disease: myasthenia gravis
Confidence: medium

Description: Myasthenia gravis is a disorder that causes weakness of the skeletal muscles, which are muscles that the body uses for movement. The weakness most often starts in the muscles around the eyes, causing drooping of the eyelids (ptosis) and difficulty coordinating eye movements, which results in blurred or double vision. In a form of the disorder called ocular myasthenia, the weakness remains confined to the eye muscles. In most people with myasthenia gravis, however, additional muscles in the face and neck are affected. Affected individuals may have unusual facial expressions, difficulty holding up 

In [ ]:
# Example 3: Autoimmune Disease
print("\n" + "=" * 70)
print("Example 3: Rheumatoid Arthritis (Autoimmune)")
print("=" * 70)

symptoms_3 = ['joint pain', 'morning stiffness', 'swollen joints', 'fatigue']
result_3 = diagnose_and_answer(symptoms_3, verbose=True)

print("\n📊 Results:")
print(f"Disease: {result_3['disease']}")
print(f"Confidence: {result_3['confidence']}")
print(f"\nDescription: {result_3['description']}")
print(f"\nRecommendation: {result_3['recommendation']}")



Example 3: Rheumatoid Arthritis (Autoimmune)

🔍 Search Strategy: COMMON_ILLNESS
✓ Common illness detected: Arthritis
  Matched 4 symptoms (score: 7)
  Using curated medical information

📋 Medical information compiled for Arthritis

📊 Results:
Disease: Arthritis
Confidence: high

Description: Arthritis is inflammation of one or more joints, causing pain, swelling, stiffness, and decreased range of motion. The two most common types are osteoarthritis (wear-and-tear damage to joint cartilage) and rheumatoid arthritis (an autoimmune condition). Symptoms often worsen with age and can significantly impact daily activities. Morning stiffness lasting more than 30 minutes is common, particularly in inflammatory types of arthritis.

Recommendation: Avoid high-impact activities that put excessive stress on affected joints, such as running or jumping. Don't remain sedentary - gentle, regular exercise is important for maintaining joint flexibility and strength. Avoid repetitive movements that stra

In [ ]:
# Example 4: Endocrine Disorder
print("\n" + "=" * 70)
print("Example 4: Hyperthyroidism (Endocrine Disorder)")
print("=" * 70)

symptoms_4 = ['weight loss', 'rapid heartbeat', 'tremor', 'sweating', 'nervousness']
result_4 = diagnose_and_answer(symptoms_4, verbose=True)

print("\n📊 Results:")
print(f"Disease: {result_4['disease']}")
print(f"Confidence: {result_4['confidence']}")
print(f"\nDescription: {result_4['description']}")
print(f"\nRecommendation: {result_4['recommendation']}")



Example 4: Hyperthyroidism (Endocrine Disorder)

🔍 Search Strategy: COMMON_ILLNESS
✓ Common illness detected: Depression or Anxiety
  Matched 3 symptoms (score: 4)
  Using curated medical information

📋 Medical information compiled for Depression or Anxiety

📊 Results:
Disease: Depression or Anxiety
Confidence: high

Description: Depression and anxiety are mental health conditions that can significantly impact daily functioning. Depression is characterized by persistent sadness, loss of interest in activities, and feelings of hopelessness. Anxiety involves excessive worry, nervousness, and physical symptoms like rapid heartbeat and muscle tension. These conditions often occur together and can affect sleep, appetite, concentration, and energy levels. Both are treatable with appropriate professional support.

Recommendation: Don't isolate yourself - maintain social connections even when you don't feel like it. Avoid self-medicating with alcohol or drugs, as these can worsen symptoms and

In [ ]:
# Example 5: Infectious Disease
print("\n" + "=" * 70)
print("Example 5: Lyme Disease (Infectious)")
print("=" * 70)

symptoms_5 = ['circular rash', 'fever', 'joint pain', 'fatigue', 'headache']
result_5 = diagnose_and_answer(symptoms_5, verbose=True)

print("\n📊 Results:")
print(f"Disease: {result_5['disease']}")
print(f"Confidence: {result_5['confidence']}")
print(f"\nDescription: {result_5['description']}")
print(f"\nRecommendation: {result_5['recommendation']}")



Example 5: Lyme Disease (Infectious)

🔍 Search Strategy: HYBRID_SEARCH
⚠ Partial match: Acute Otitis Media (score: 2)
  Comparing with BioGPT search...
  BioGPT suggested: 'systemic lupus erythematosus (SLE)' → 'Lupus'
  Full search result: Lupus
  → Using full search result: Lupus
  Extracting from MedQuAD...

📋 Medical information compiled for Lupus

📊 Results:
Disease: Lupus
Confidence: medium

Description: Lupus is an autoimmune disease that can affect almost every organ in the body. Symptoms of lupus can range from very mild to life-threatening. There are three types of lupus; systemic lupus erythematosus, discoid lupus, and drug-induced lupus. Genetics is thought to play a role in the development of lupus along with other lifestyle and environmental factors. Studies suggest that a number of different genes may be involved in determining a persons likelihood of developing the disease, which tissues and organs are affected, and the severity of disease.

Recommendation: How might l

In [ ]:
# Example 6: Neurological Disorder
print("\n" + "=" * 70)
print("Example 6: Multiple Sclerosis (Neurological)")
print("=" * 70)

symptoms_6 = ['vision problems', 'numbness', 'tingling', 'weakness', 'balance problems']
result_6 = diagnose_and_answer(symptoms_6, verbose=True)

print("\n📊 Results:")
print(f"Disease: {result_6['disease']}")
print(f"Confidence: {result_6['confidence']}")
print(f"\nDescription: {result_6['description']}")
print(f"\nRecommendation: {result_6['recommendation']}")



Example 6: Multiple Sclerosis (Neurological)

🔍 Search Strategy: FULL_MEDQUAD
  No common illness match.
  Using BioGPT for diagnosis...
  BioGPT diagnosis: 'multiple sclerosis (MS)'
  Matched to MedQuAD: 'Multiple Sclerosis'
  Extracting information from MedQuAD...

📋 Medical information compiled for Multiple Sclerosis

📊 Results:
Disease: Multiple Sclerosis
Confidence: medium

Description: Multiple sclerosis (MS) is a nervous system disease that affects your brain and spinal cord. It damages the myelin sheath, the material that surrounds and protects your nerve cells. This damage slows down or blocks messages between your brain and your body, leading to the symptoms of MS. They can include - Visual disturbances - Muscle weakness - Trouble with coordination and balance - Sensations such as numbness, prickling, or "pins and needles" - Thinking and memory problems No one knows what causes MS. It may be an autoimmune disease, which happens when your immune system attacks healthy cells i

In [ ]:
# Example 2: Rare/specific condition - Using BioGPT Diagnosis
print("\n" + "=" * 60)
print("Example 2: Rare Condition (BioGPT Diagnosis)")
print("=" * 60)

patient_symptoms_2 = [
    'muscle weakness',
    'difficulty swallowing',
    'double vision'
]

# Use BioGPT for diagnosis
result_2 = diagnose_and_answer(patient_symptoms_2, use_biogpt=True, verbose=True)
print("\n📊 Results:")
print(f"Disease: {result_2['disease']}")
print(f"Confidence: {result_2['confidence']}")
print(f"\nDescription: {result_2['description']}")
print(f"\nRecommendation: {result_2['recommendation']}")


Example 2: Rare Condition (BioGPT Diagnosis)

🔍 Search Strategy: FULL_MEDQUAD
  No common illness match.
  Using BioGPT for diagnosis...
  BioGPT diagnosis: 'myasthenia gravis'
  Matched to MedQuAD: 'myasthenia gravis'
  Extracting information from MedQuAD...

📋 Medical information compiled for myasthenia gravis

📊 Results:
Disease: myasthenia gravis
Confidence: medium

Description: Myasthenia gravis is a disorder that causes weakness of the skeletal muscles, which are muscles that the body uses for movement. The weakness most often starts in the muscles around the eyes, causing drooping of the eyelids (ptosis) and difficulty coordinating eye movements, which results in blurred or double vision. In a form of the disorder called ocular myasthenia, the weakness remains confined to the eye muscles. In most people with myasthenia gravis, however, additional muscles in the face and neck are affected. Affected individuals may have unusual facial expressions, difficulty holding up the head, 

In [ ]:
# Example 3: Diabetes symptoms (should detect common illness)
print("\n" + "=" * 60)
print("Example 3: Diabetes Symptoms")
print("=" * 60)

patient_symptoms_3 = [
    'increased thirst',
    'frequent urination',
    'blurred vision',
    'fatigue'
]

result_3 = diagnose_and_answer(patient_symptoms_3, verbose=True)
print("\n📊 Results:")
print(f"Disease: {result_3['disease']}")
print(f"Confidence: {result_3['confidence']}")
print(f"\nDescription: {result_3['description']}")
print(f"\nRecommendation: {result_3['recommendation']}")


Example 3: Diabetes Symptoms

🔍 Search Strategy: COMMON_ILLNESS
✓ Common illness detected: Diabetes
  Matched 4 symptoms (score: 7)
  Using curated medical information

📋 Medical information compiled for Diabetes

📊 Results:
Disease: Diabetes
Confidence: high

Description: Diabetes is a chronic condition that affects how your body processes blood sugar (glucose). In Type 1 diabetes, the body doesn't produce insulin. In Type 2 diabetes, the body doesn't use insulin properly. Common symptoms include increased thirst, frequent urination, extreme hunger, unexplained weight loss, fatigue, and blurred vision. If left unmanaged, diabetes can lead to serious complications affecting the heart, kidneys, eyes, and nerves.

Recommendation: Avoid foods high in refined sugars and simple carbohydrates, which can cause blood sugar spikes. Don't skip meals or go long periods without eating, as this can cause dangerous blood sugar fluctuations. Avoid excessive alcohol consumption, which can affect bloo

## 15 Test Cases for Accuracy Evaluation

These tests cover common conditions that the chatbot should handle well:
- Common illnesses (Tier 1 fast path)
- Clear symptom patterns
- Well-documented diseases in MedQuAD

In [21]:
# Test Suite: 15 Cases for Disease Identification Accuracy

test_cases = [
    {
        "test_id": 1,
        "description": "Common Cold",
        "symptoms": ["runny nose", "sore throat", "cough", "sneezing"],
        "expected_disease": "Upper Respiratory Tract Infection"
    },
    {
        "test_id": 2,
        "description": "Type 2 Diabetes",
        "symptoms": ["increased thirst", "frequent urination", "fatigue", "blurred vision"],
        "expected_disease": "Diabetes"
    },
    {
        "test_id": 3,
        "description": "Hypertension",
        "symptoms": ["headache", "dizziness", "shortness of breath"],
        "expected_disease": "Hypertension"
    },
    {
        "test_id": 4,
        "description": "Asthma",
        "symptoms": ["wheezing", "shortness of breath", "chest tightness", "coughing"],
        "expected_disease": "Asthma"
    },
    {
        "test_id": 5,
        "description": "Migraine",
        "symptoms": ["severe headache", "nausea", "sensitivity to light", "visual disturbances"],
        "expected_disease": "Migraine"
    },
    {
        "test_id": 6,
        "description": "Influenza",
        "symptoms": ["fever", "body aches", "fatigue", "cough", "sore throat"],
        "expected_disease": "Influenza"
    },
    {
        "test_id": 7,
        "description": "Allergies",
        "symptoms": ["sneezing", "itchy eyes", "runny nose", "congestion"],
        "expected_disease": "Allergies"
    },
    {
        "test_id": 8,
        "description": "Gastroenteritis",
        "symptoms": ["diarrhea", "nausea", "vomiting", "stomach cramps"],
        "expected_disease": "Gastroenteritis"
    },
    {
        "test_id": 9,
        "description": "Depression or Anxiety",
        "symptoms": ["nervousness", "rapid heartbeat", "sweating", "restlessness"],
        "expected_disease": "Depression or Anxiety"
    },
    {
        "test_id": 17,
        "description": "Myasthenia Gravis",
        "symptoms": ["muscle weakness", "drooping eyelids", "double vision", "difficulty swallowing"],
        "expected_disease": "Myasthenia Gravis"
    },
    {
        "test_id": 11,
        "description": "Arthritis",
        "symptoms": ["joint pain", "stiffness", "swelling", "reduced range of motion"],
        "expected_disease": "Arthritis"
    },
    {
        "test_id": 12,
        "description": "Bronchitis",
        "symptoms": ["persistent cough", "mucus production", "chest discomfort", "fatigue"],
        "expected_disease": "Bronchitis"
    },
    {
        "test_id": 13,
        "description": "Urinary Tract Infection",
        "symptoms": ["painful urination", "frequent urination", "lower abdominal pain"],
        "expected_disease": "Urinary Tract Infection"
    },
    {
        "test_id": 14,
        "description": "Hypothyroidism",
        "symptoms": ["fatigue", "weight gain", "cold sensitivity", "dry skin"],
        "expected_disease": "Hypothyroidism"
    },
    {
        "test_id": 15,
        "description": "Pneumonia",
        "symptoms": ["cough", "fever", "chest pain", "difficulty breathing"],
        "expected_disease": "Pneumonia"
    }
]

print(f"Created {len(test_cases)} test cases")
print("\nTest cases:")
for tc in test_cases:
    print(f"  {tc['test_id']}. {tc['description']} - {len(tc['symptoms'])} symptoms")


Created 15 test cases

Test cases:
  1. Common Cold - 4 symptoms
  2. Type 2 Diabetes - 4 symptoms
  3. Hypertension - 3 symptoms
  4. Asthma - 4 symptoms
  5. Migraine - 4 symptoms
  6. Influenza - 5 symptoms
  7. Allergies - 4 symptoms
  8. Gastroenteritis - 4 symptoms
  9. Depression or Anxiety - 4 symptoms
  17. Myasthenia Gravis - 4 symptoms
  11. Arthritis - 4 symptoms
  12. Bronchitis - 4 symptoms
  13. Urinary Tract Infection - 3 symptoms
  14. Hypothyroidism - 4 symptoms
  15. Pneumonia - 4 symptoms


In [22]:
# Run all 15 tests and collect results

import time
from datetime import datetime

print("=" * 80)
print(f"STARTING TEST SUITE - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 80)

results = []

for test in test_cases:
    print(f"\n{'='*80}")
    print(f"Test {test['test_id']}/{len(test_cases)}: {test['description']}")
    print(f"Symptoms: {', '.join(test['symptoms'])}")
    print(f"Expected: {test['expected_disease']}")
    print("-" * 80)
    
    # Run diagnosis
    start_time = time.time()
    result = diagnose_and_answer(test['symptoms'], use_biogpt=True, verbose=False)
    elapsed = time.time() - start_time
    
    # Store results
    predicted = result['disease']
    correct = predicted.lower() == test['expected_disease'].lower()
    
    results.append({
        'test_id': test['test_id'],
        'description': test['description'],
        'expected': test['expected_disease'],
        'predicted': predicted,
        'correct': correct,
        'time': elapsed,
        'strategy': result.get('strategy', 'unknown')
    })
    
    # Print result
    status = "✅ CORRECT" if correct else "❌ INCORRECT"
    print(f"Predicted: {predicted}")
    print(f"Status: {status}")
    print(f"Time: {elapsed:.2f}s")
    print(f"Strategy: {result.get('strategy', 'unknown')}")

print(f"\n{'='*80}")
print("TEST SUITE COMPLETED")
print("=" * 80)


STARTING TEST SUITE - 2025-11-21 23:27:54

Test 1/15: Common Cold
Symptoms: runny nose, sore throat, cough, sneezing
Expected: Upper Respiratory Tract Infection
--------------------------------------------------------------------------------
Predicted: Upper Respiratory Tract Infection
Status: ✅ CORRECT
Time: 0.00s
Strategy: common_illness

Test 2/15: Type 2 Diabetes
Symptoms: increased thirst, frequent urination, fatigue, blurred vision
Expected: Diabetes
--------------------------------------------------------------------------------
Predicted: Diabetes
Status: ✅ CORRECT
Time: 0.00s
Strategy: common_illness

Test 3/15: Hypertension
Symptoms: headache, dizziness, shortness of breath
Expected: Hypertension
--------------------------------------------------------------------------------
Predicted: Hypertension
Status: ✅ CORRECT
Time: 0.00s
Strategy: common_illness

Test 4/15: Asthma
Symptoms: wheezing, shortness of breath, chest tightness, coughing
Expected: Asthma
---------------------

In [23]:
# Calculate and Display Accuracy Metrics

correct_count = sum(1 for r in results if r['correct'])
total_count = len(results)
accuracy = (correct_count / total_count) * 100

print("\n" + "=" * 80)
print("ACCURACY REPORT")
print("=" * 80)

print(f"\n📊 Overall Accuracy: {correct_count}/{total_count} ({accuracy:.1f}%)")

# Break down by strategy
strategy_stats = {}
for r in results:
    strat = r['strategy']
    if strat not in strategy_stats:
        strategy_stats[strat] = {'correct': 0, 'total': 0}
    strategy_stats[strat]['total'] += 1
    if r['correct']:
        strategy_stats[strat]['correct'] += 1

print(f"\n📈 Performance by Strategy:")
for strategy, stats in strategy_stats.items():
    strat_acc = (stats['correct'] / stats['total']) * 100
    print(f"  {strategy}: {stats['correct']}/{stats['total']} ({strat_acc:.1f}%)")

# Show incorrect predictions
incorrect = [r for r in results if not r['correct']]
if incorrect:
    print(f"\n❌ Incorrect Predictions ({len(incorrect)}):")
    for r in incorrect:
        print(f"  Test {r['test_id']}: Expected '{r['expected']}', Got '{r['predicted']}'")
else:
    print(f"\n🎉 Perfect Score! All tests passed!")

# Timing statistics
avg_time = sum(r['time'] for r in results) / len(results)
min_time = min(r['time'] for r in results)
max_time = max(r['time'] for r in results)

print(f"\n⏱️  Timing Statistics:")
print(f"  Average: {avg_time:.2f}s")
print(f"  Min: {min_time:.2f}s")
print(f"  Max: {max_time:.2f}s")

print("\n" + "=" * 80)



ACCURACY REPORT

📊 Overall Accuracy: 9/15 (60.0%)

📈 Performance by Strategy:
  common_illness: 6/7 (85.7%)
  biogpt: 3/8 (37.5%)

❌ Incorrect Predictions (6):
  Test 6: Expected 'Influenza', Got 'Upper Respiratory Tract Infection'
  Test 8: Expected 'Gastroenteritis', Got 'Celiac Disease'
  Test 11: Expected 'Arthritis', Got 'rheumatoid arthritis'
  Test 12: Expected 'Bronchitis', Got 'Asthma'
  Test 14: Expected 'Hypothyroidism', Got 'Sjogren's Syndrome'
  Test 15: Expected 'Pneumonia', Got 'Tuberculosis'

⏱️  Timing Statistics:
  Average: 31.48s
  Min: 0.00s
  Max: 222.65s



In [ ]:
# Detailed Results Table

import pandas as pd

# Create DataFrame from results
results_df = pd.DataFrame(results)

# Format for display
display_df = results_df[['test_id', 'description', 'expected', 'predicted', 'correct', 'time', 'strategy']].copy()
display_df['time'] = display_df['time'].round(2)
display_df.columns = ['Test #', 'Description', 'Expected', 'Predicted', 'Correct?', 'Time (s)', 'Strategy']

print("\nDetailed Test Results:")
print(display_df.to_string(index=False))

# Save results to CSV
results_df.to_csv('test_results.csv', index=False)
print("\n💾 Results saved to 'test_results.csv'")
